In [2]:
import os
import cv2
import numpy as np
import pulse2percept as p2p
from pulse2percept.implants import AlphaAMS

In [3]:
def kmeans_segment(image, K=3):
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
    h, w, c = image.shape
    pixels = image_lab.reshape(-1, 3).astype(np.float32)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1.0)

    _, labels, centers = cv2.kmeans(
        pixels,
        K,
        None,
        criteria,
        10,
        cv2.KMEANS_PP_CENTERS
    )

    centers = np.uint8(centers)
    segmented = centers[labels.flatten()]
    segmented = segmented.reshape((h, w, 3))
    segmented_bgr = cv2.cvtColor(segmented, cv2.COLOR_Lab2BGR)

    return segmented_bgr
    
def preprocess_for_percept(img):
    # grayscale
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # normalization
    img = img.astype(np.float32) / 255.0
    return img

In [4]:
# file directories
input_video_dir = "/Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/Videos"
output_base_dir = "/Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output"

os.makedirs(output_base_dir, exist_ok=True)

# pulse2percept
model = p2p.models.ScoreboardModel(xrange=(-7,7), yrange=(-7,7), xystep=0.1)
model.build()
implant = AlphaAMS()

# process videos
video_files = [f for f in os.listdir(input_video_dir) if f.endswith(".mp4")]

for video_file in video_files:
    video_path = os.path.join(input_video_dir, video_file)
    print(f"[INFO] Processing {video_file}")

    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30
    frame_idx = 0

    video_name = os.path.splitext(video_file)[0]
    segVisDir = os.path.join(output_base_dir, video_name, "segmented")
    perceptDir = os.path.join(output_base_dir, video_name, "percepts")

    os.makedirs(segVisDir, exist_ok=True)
    os.makedirs(perceptDir, exist_ok=True)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # classic segmentation
        segmented = kmeans_segment(frame, K=3)

        # save segmented frame 
        seg_vis_path = os.path.join(segVisDir, f"frame_{frame_idx:05d}.png")
        cv2.imwrite(seg_vis_path, segmented)

        # normalization
        preprocessed = preprocess_for_percept(segmented)

        # creating percept
        stimulus = p2p.stimuli.ImageStimulus(preprocessed)
        implant.stim = stimulus.resize(implant.shape)
        percept = model.predict_percept(implant)
        percept_path = os.path.join(perceptDir, f"frame_{frame_idx:05d}.png")
        percept.save(percept_path)

        frame_idx += 1

    cap.release()

    # recombining frames to percept video
    percept_files = sorted(os.listdir(perceptDir))
    if len(percept_files) == 0:
        print(f"[WARNING] No percept frames for {video_file}")
        continue

    first_frame = cv2.imread(os.path.join(perceptDir, percept_files[0]))
    height, width, _ = first_frame.shape
    output_video_path = os.path.join(output_base_dir, f"{video_name}_percept.mp4")

    out = cv2.VideoWriter(output_video_path,
                          cv2.VideoWriter_fourcc(*"mp4v"),
                          fps,
                          (width, height))

    for f in percept_files:
        frame = cv2.imread(os.path.join(perceptDir, f))
        out.write(frame)

    out.release()
    print(f"Percept video saved to: {output_video_path}")

print("All videos processed!")


[INFO] Processing North-East.mp4


Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/North-East/percepts/frame_00000.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/North-East/percepts/frame_00001.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/North-East/percepts/frame_00002.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/North-East/percepts/frame_00003.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/North-East/percepts/frame_00004.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/North-East/percepts/frame_00005.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/North-East/percepts/frame_00006.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/North-East/percepts/frame_00007.png.


Percept video saved to: /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/North-East_percept.mp4
[INFO] Processing road-video-dubai.mp4


Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/road-video-dubai/percepts/frame_00000.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/road-video-dubai/percepts/frame_00001.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/road-video-dubai/percepts/frame_00002.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/road-video-dubai/percepts/frame_00003.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/road-video-dubai/percepts/frame_00004.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/road-video-dubai/percepts/frame_00005.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/road-video-dubai/percepts/frame_00006.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_ou

Percept video saved to: /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/road-video-dubai_percept.mp4
[INFO] Processing pplWalk1.mp4


...

Percept video saved to: /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/pplWalk1_percept.mp4
[INFO] Processing pplWalk2.mp4


Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/pplWalk2/percepts/frame_00000.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/pplWalk2/percepts/frame_00001.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/pplWalk2/percepts/frame_00002.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/pplWalk2/percepts/frame_00003.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/pplWalk2/percepts/frame_00004.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/pplWalk2/percepts/frame_00005.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/pplWalk2/percepts/frame_00006.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/pplWalk2/percepts/frame_00007.png.
Created /Users/n

Percept video saved to: /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/pplWalk2_percept.mp4
[INFO] Processing night1.mp4


Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/night1/percepts/frame_00000.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/night1/percepts/frame_00001.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/night1/percepts/frame_00002.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/night1/percepts/frame_00003.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/night1/percepts/frame_00004.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/night1/percepts/frame_00005.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/night1/percepts/frame_00006.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/night1/percepts/frame_00007.png.
Created /Users/nimrahs/Desktop/S

Percept video saved to: /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/classic/classic_output/night1_percept.mp4
All videos processed!
